In [1]:
import pandas as pd
import numpy as np
from skimpy import clean_columns
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Climate Data

In [2]:
climate = pd.read_csv("Data/2023_scm_climate.csv")
climate = clean_columns(climate)
climate["date"] = pd.to_datetime(climate["date"], format="%m/%d/%y")
climate["week"] = climate["date"].dt.strftime("%U").astype(int)
#climate

# Weekly Climate Data
Average temperatures and total rain.

In [3]:
weekly_climate = (
    climate[
        ["record_id", "week", "min_temp_f", "ave_temp_f", "max_temp_f"]
    ]
    .groupby(["record_id", "week"])
    .mean()
)
weekly_climate = weekly_climate.reset_index()
weekly_climate = clean_columns(weekly_climate)
weekly_climate["next_week"] = weekly_climate["week"] + 1

weekly_rain = (climate[
        ["record_id", "week", "rain_inches"]
    ]
    .groupby(["record_id", "week"])
    .sum()
)

weekly_rain = weekly_rain.reset_index()
weekly_rain = clean_columns(weekly_rain)

weekly_climate = weekly_climate.merge(weekly_rain, on=['record_id', 'week'])
#weekly_climate


# Soil Data

In [14]:
soil = pd.read_csv("Data/2023_scm_soil.csv")
soil = clean_columns(soil)
#soil

# Management Data

In [5]:
management = pd.read_csv("Data/2023_management.csv")
management = clean_columns(management)
#management

# Wire Mesh Data

In [6]:
wire_mesh = pd.read_csv("Data/2023_wire_mesh.csv")
wire_mesh = clean_columns(wire_mesh)
wire_mesh["number_wireworm"] = (
    wire_mesh["number_wireworm"].replace(">10", 10).astype(float)
)
wire_mesh["total_phorid_maggots"] = (
    wire_mesh["total_phorid_maggots"].replace(">10", 10).astype(float)
)
wire_mesh["total_phorid_pupae"] = (
    wire_mesh["total_phorid_pupae"].replace(">10", 10).astype(float)
)

wire_mesh = wire_mesh.loc[wire_mesh['project'] == 'R']
#wire_mesh

# Wire Mesh Intake

In [7]:
wire_mesh_intake = pd.read_csv("Data/2023_wire_mesh_intake.csv")
wire_mesh_intake = clean_columns(wire_mesh_intake)
wire_mesh_intake = wire_mesh_intake.loc[wire_mesh_intake["project"] == "R"]
#wire_mesh_intake

In [8]:
wire_mesh = wire_mesh.merge(
    wire_mesh_intake[
        ["container_number", "container_type", "collection_date", "record_id", 'project']
    ],
    on=["container_number", 'project'],
    how="left",
)
#wire_mesh


In [9]:
bean_containers = wire_mesh[wire_mesh["seed_type"] == "Bean"]
bean_containers = bean_containers["container_number"]
#bean_containers

bean_data = wire_mesh[wire_mesh["container_number"].isin(bean_containers)]
bean_data = bean_data.loc[bean_data["seed_type"] == "Total"]
bean_data.loc[:, "seed_type"] = "Bean"
#bean_data

In [10]:
corn_containers = wire_mesh[wire_mesh["seed_type"] == "Corn"]
corn_containers = corn_containers["container_number"]
#corn_containers

corn_data = wire_mesh[wire_mesh["container_number"].isin(corn_containers)]
corn_data = corn_data.loc[corn_data["seed_type"] == "Total"]
corn_data.loc[:, "seed_type"] = "Corn"
#corn_data

In [11]:
wire_mesh = pd.concat([bean_data, corn_data], axis=0)
#wire_mesh = wire_mesh.loc[wire_mesh["project"] == "R"]
wire_mesh["collection_date"] = pd.to_datetime(
    wire_mesh["collection_date"], format="%m/%d/%y"
)
wire_mesh

,id,container_number,project,seed_type,number_seeds,scm_seeds,other_seeds,total_scm_maggots,total_scm_pupae,number_wireworm,phorid_seeds,total_phorid_maggots,total_phorid_pupae,notes,container_type,collection_date,record_id
2,3,3011,R,Bean,8.0,1.0,2.0,1.0,2,0.0,1.0,2.0,3.0,NaN,deli cup,2023-04-17,GAB_ALL
8,9,3068,R,Bean,8.0,0.0,0.0,0.0,5,1.0,2.0,7.0,9.0,NaN,deli cup,2023-04-18,SMI_JOH_2
14,15,3010,R,Bean,10.0,0.0,1.0,0.0,0,0.0,0.0,0.0,19.0,NaN,deli cup,2023-04-17,GAB_STE
20,21,3003,R,Bean,5.0,0.0,1.0,0.0,17,0.0,0.0,0.0,0.0,NaN,deli cup,2023-04-18,DIP_CUR
29,30,3042,R,Bean,10.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,NaN,deli cup,2023-04-17,SMI_MCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,5085,5100,R,Corn,10.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,deli cup,2023-06-09,MIL_MCC
827,5097,5114,R,Corn,10.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,deli cup,2023-06-12,SMI_KEL
830,5103,5116,R,Corn,10.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,deli cup,2023-06-13,SMI_JOH_2
833,5184,5033,R,Corn,10.0,0.0,3.0,0.0,0,0.0,0.0,0.0,1.0,NaN,deli cup,2023-06-06,DIP_FLE


In [12]:
wire_mesh

,id,container_number,project,seed_type,number_seeds,scm_seeds,other_seeds,total_scm_maggots,total_scm_pupae,number_wireworm,phorid_seeds,total_phorid_maggots,total_phorid_pupae,notes,container_type,collection_date,record_id
2,3,3011,R,Bean,8.0,1.0,2.0,1.0,2,0.0,1.0,2.0,3.0,NaN,deli cup,2023-04-17,GAB_ALL
8,9,3068,R,Bean,8.0,0.0,0.0,0.0,5,1.0,2.0,7.0,9.0,NaN,deli cup,2023-04-18,SMI_JOH_2
14,15,3010,R,Bean,10.0,0.0,1.0,0.0,0,0.0,0.0,0.0,19.0,NaN,deli cup,2023-04-17,GAB_STE
20,21,3003,R,Bean,5.0,0.0,1.0,0.0,17,0.0,0.0,0.0,0.0,NaN,deli cup,2023-04-18,DIP_CUR
29,30,3042,R,Bean,10.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,NaN,deli cup,2023-04-17,SMI_MCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,5085,5100,R,Corn,10.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,deli cup,2023-06-09,MIL_MCC
827,5097,5114,R,Corn,10.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,deli cup,2023-06-12,SMI_KEL
830,5103,5116,R,Corn,10.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,deli cup,2023-06-13,SMI_JOH_2
833,5184,5033,R,Corn,10.0,0.0,3.0,0.0,0,0.0,0.0,0.0,1.0,NaN,deli cup,2023-06-06,DIP_FLE
